In [ ]:
import numpy as np
import pandas as pd
import math, os, pathlib
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
dfMerge = pd.read_csv('~/Downloads/Brain_GSE50161.csv')
dfMerge

In [ ]:
dfMerge.info()

In [ ]:
dfMerge.shape

In [ ]:
dfMerge.describe()

In [ ]:
dfMerge.type.unique()

In [ ]:
dfMerge['type'].value_counts()

In [ ]:
df_count = dfMerge['type'].value_counts()

In [ ]:
df_classes = pd.DataFrame({'classes':df_count.index, 'Frequency':df_count.values})

In [ ]:
x_classes = df_classes['classes']
y_Frequency = df_classes['Frequency']

In [ ]:
plt.bar(x_classes, y_Frequency, color ='maroon',width = 0.9)
plt.xlabel('classes')
plt.ylabel('Frequency')
plt.xticks(rotation=50)

In [ ]:
fig, ax = plt.subplots()
width = 0.9
ind = np.arange(len(y_Frequency))
 
ax.barh(x_classes, y_Frequency, width, color = "maroon")
 
for i, v in enumerate(y_Frequency):
    ax.text(v + 3, i + .25, str(v),
            color = 'black', fontweight = 'bold')
plt.xlabel('Frequency')
plt.ylabel('Classes')
plt.show()

In [ ]:
plt.barh(x_classes, y_Frequency)
 
for index, value in enumerate(y_Frequency):
    plt.text(value, index,
             str(value))

plt.xlabel('Frequency')
plt.ylabel('Classes')
plt.show()

In [ ]:
dfMerge.isnull().sum()

In [ ]:
dfMerge.columns

In [ ]:
dfMerge.type.unique()

In [ ]:
Target = {k: v for k, v in zip(['ependymoma', 'glioblastoma', 'medulloblastoma', 'pilocytic_astrocytoma', 'normal'], list(range(len(['ependymoma', 'glioblastoma', 'medulloblastoma', 'pilocytic_astrocytoma', 'normal']))))}
Target

In [ ]:
for i in range(dfMerge.shape[0]):
    dfMerge.iloc[i, 1] = Target[dfMerge.iloc[i, 1]]

In [ ]:
dfMerge

In [ ]:
dfMerge['type'].value_counts()

In [ ]:
X = np.array(dfMerge.iloc[:, 2:])
y = np.asarray(dfMerge.iloc[:, 1]).astype('int64')

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
def GELU(x):
    res = 0.5 * x * (1 + tf.nn.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * (x ** 3))))
    return res


class ResMLP(tf.keras.layers.Layer):
    def __init__(self, units, activation, residual_path):
        super(ResMLP, self).__init__()
        self.residual_path = residual_path
        self.D1 = Dense(units, activation=activation)
        self.D2 = Dense(units, activation=activation)

        if self.residual_path:
            self.D3 = Dense(units)
            self.D4 = Dense(units)

    def call(self, inputs):
        residual = inputs

        x = self.D1(inputs)
        y = self.D2(x)

        if self.residual_path:
            residual = self.D3(inputs)
            residual = GELU(residual)
            residual = self.D4(residual)
            residual = GELU(residual)

        output = y + residual
        return output

In [ ]:
class NetWork(tf.keras.Model):
    def __init__(self, initial_filters, block_list):
        super(NetWork, self).__init__()
        self.initial_filters = initial_filters
        self.block_list = block_list

        self.D1 = Dense(self.initial_filters, activation='relu')
        self.B1 = BatchNormalization()

        self.blocks = tf.keras.models.Sequential()
        for block_id in range(len(block_list)):
            for layer_id in range(block_list[block_id]):
                if block_id != 0 and layer_id == 0:
                    block = ResMLP(units=self.initial_filters, activation='relu', residual_path=True)
                else:
                    block = ResMLP(units=self.initial_filters, activation='relu', residual_path=False)
                self.blocks.add(block)
            self.initial_filters *= 2
        self.D2 = Dense(5, activation='softmax')


    def call(self, inputs):
        x = self.D1(inputs)
        x = self.B1(x)
        x = self.blocks(x)
        y = self.D2(x)
        return y

In [ ]:
net = NetWork(initial_filters=1024, block_list=[3, 3, 3])

net.compile(optimizer='adam',
           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
           metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = './checkpoint/Baseline.ckpt'
if os.path.exists(checkpoint_save_path + '.index'):
    net.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path, save_weights_only=True,
                                                 save_best_only=True)

history = net.fit(X, y, epochs=50, batch_size=16, callbacks=[cp_callback])

net.summary()

file = open('./weights.txt', 'w')
for v in net.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')

file.close()

In [ ]:
acc = history.history['sparse_categorical_accuracy']
loss = history.history['loss']

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(acc)
plt.title('Training Acc')

plt.subplot(1, 2, 2)
plt.plot(loss)
plt.title('Training Loss')

plt.show()